#Imports

In [ ]:
# File saved on google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import requests
import sys
import json
from datetime import datetime, timedelta
import holidays

np.random.seed(42)
import scipy
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

#Data Modelling

####Import Cleaned CSV

In [ ]:
data_path = "/content/drive/MyDrive/Project/data/final_trip_feature_data.csv"
df_full_non_zero_net_demand_trim = pd.read_csv(data_path)

In [ ]:
df_full_non_zero_net_demand_trim = df_full_non_zero_net_demand_trim.rename(columns={
    "Start Station Id" : "Station Id",
    "Binned_Start_Time" : "Time",
    "capacity" : "Station Capacity",
    "nearest_transit" : "Nearest Transit Station Name",
    "POI_distance" : "Distance to Nearest Transit Station",
    "Is_Weekend" : "Weekend",
    "Active_TO" : "Active TO",
    "Lockdown_Stay_at_Home" : "Lockdown"
})

In [ ]:
df_full_non_zero_net_demand_trim[['Active TO','Lockdown','Holiday','Weekend']] = df_full_non_zero_net_demand_trim[['Active TO','Lockdown','Holiday','Weekend']].astype(pd.SparseDtype("int64",0))

In [ ]:
df_full_non_zero_net_demand_trim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1912438 entries, 0 to 1912437
Data columns (total 18 columns):
 #   Column                               Dtype           
---  ------                               -----           
 0   Station Id                           int64           
 1   Time                                 object          
 2   Station Capacity                     int64           
 3   Nearest Transit Station Name         object          
 4   Distance to Nearest Transit Station  float64         
 5   Temp (°C)                            float64         
 6   Precip. Amount (mm)                  float64         
 7   Wind Spd (km/h)                      float64         
 8   Wind Dir (10s deg)                   float64         
 9   Visibility (km)                      float64         
 10  Holiday                              Sparse[int64, 0]
 11  Weekend                              Sparse[int64, 0]
 12  Year                                 int64           
 1

In [ ]:
# df_full_non_zero_net_demand_trim = df_full_non_zero_net_demand_trim.sample(frac = 0.2, random_state = 42)

In [ ]:
df_full_non_zero_net_demand_trim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1912438 entries, 0 to 1912437
Data columns (total 18 columns):
 #   Column                               Dtype           
---  ------                               -----           
 0   Station Id                           int64           
 1   Time                                 object          
 2   Station Capacity                     int64           
 3   Nearest Transit Station Name         object          
 4   Distance to Nearest Transit Station  float64         
 5   Temp (°C)                            float64         
 6   Precip. Amount (mm)                  float64         
 7   Wind Spd (km/h)                      float64         
 8   Wind Dir (10s deg)                   float64         
 9   Visibility (km)                      float64         
 10  Holiday                              Sparse[int64, 0]
 11  Weekend                              Sparse[int64, 0]
 12  Year                                 int64           
 1

### Train Test Split

In [ ]:
X = df_full_non_zero_net_demand_trim.drop("Positive Net Demand", axis = 1)
Y = df_full_non_zero_net_demand_trim["Positive Net Demand"].copy()

In [ ]:
X_train_pre_pipeline, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 123, stratify = Y)

## Pipeline Creation

In [ ]:
# This class is defined in the textbook and simply restricts the dataframe to specific features
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(['Station Capacity','Distance to Nearest Transit Station', 'Temp (°C)', 'Precip. Amount (mm)', 'Wind Spd (km/h)',
       'Wind Dir (10s deg)', 'Visibility (km)', 'Month','Holiday', 'Weekend', 'Active TO', 'Lockdown'])),
        ('std_scaler', RobustScaler())
    ])

In [ ]:
cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(['Time', 'Nearest Transit Station Name'])),
        ("cat_encoder", OneHotEncoder(categories='auto')),
    ])

In [ ]:
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [ ]:
X_train = preprocess_pipeline.fit_transform(X_train_pre_pipeline)

In [ ]:
X_train.shape

(1529950, 78)

# PCA

In [ ]:
X_train =X_train.toarray()

In [ ]:
pca = PCA(n_components = 0.95 )
X_reduced = pca.fit_transform(X_train)

In [ ]:
X_reduced.shape

(1529950, 28)

In [ ]:
# (PCA) Transform Test Set here:

#Random Forest

In [ ]:
# #%%time
# forest_PCA = RandomForestClassifier(n_estimators=100, random_state=0)
# accuracy = forest_PCA.fit(X_reduced,Y_train)

In [1]:
# for rank,mean_score, params in zip(KN_grid_search_results['rank_test_score'],KN_grid_search_results['mean_test_score'], KN_grid_search_results['params']):
#     print(rank,mean_score, params)

NameError: ignored

In [ ]:
# def plot_roc_curve(fpr, tpr, label = None):
#     plt.plot(fpr, tpr, label = label)
#     plt.plot([0, 1], [0, 1], 'k--')
#     #plt.axis([0, 1, 0, 1])
#     plt.xlabel('False Positive Rate', fontsize=16)
#     plt.ylabel('True Positive Rate', fontsize=16)
#     plt.legend(loc="lower right", fontsize=16)